In [ ]:
import torch, torchvision
import os
import random
import logging #Se renseigner, par encore utilisé
import dataset
import metrics

import constants as cst
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

import loss_fn
from unet import UNET

In [ ]:
random.seed(cst.SEED)
torch.manual_seed(cst.SEED)
np.random.seed(cst.SEED)

def predict_img(model, image, device, transform, out_threshold=0.5):
    with torch.no_grad():
        x = image
        logits = model(x.to(device))
        logits = transform(logits)
        y_pred = nn.Softmax(dim=1)(logits)
        proba = y_pred.detach().cpu().squeeze(0).numpy()[1, :, :]
        return proba > out_threshold

In [ ]:
TERM = "BR1A"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/br1a"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "BR1B"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/br1b"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "BR2A"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/br2a"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "BR2B"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/br2b"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "CB1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/cb1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "CB2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/cb2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "CH1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/ch1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "CH2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/ch2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "CL1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/cl1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "CL2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/cl2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "D1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/d1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "D2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/d2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "EN1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/en1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "EN2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/en2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "HM1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/hm1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "HM2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/hm2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "M1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/m1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "M2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/m2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "N"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/n"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [1]:
TERM = "OC1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/oc1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

NameError: name 'torch' is not defined

In [ ]:
TERM = "OC2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/oc2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "OP1"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/op1"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "OP2"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/op2"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "P"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))

In [ ]:
TERM = "VC"
EXTRA = "ALL"

DATASET = "/notebooks/images"
MASKS = "/notebooks/vc"

SIZE = (384, 512)

DEVICE_NAME = "cpu"
if torch.cuda.is_available():
    DEVICE_NAME = 'cuda:0'
DEVICE = torch.device(DEVICE_NAME)

transform = transforms.Compose([transforms.Resize(SIZE),
                                transforms.Pad((0, 64, 0, 64))])
untransform = transforms.Compose([transforms.CenterCrop(SIZE),
                                 transforms.Resize((1932, 2576))])

model = UNET(3, 2)
overall_best_model = UNET(3, 2)
overall_best_model = model
model.to(DEVICE)

best_fold = 0
overall_best = 10
best_fold_epoch = 0
fold_best_vals = []

image_folder = DATASET
mask_folder = MASKS


for fold in range(cst.FOLDS):
    # Datasets and loaders
    training_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                  mask_folder,
                                                  actual_fold=fold,
                                                  dataset="train",
                                                  folds=5)
    validation_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                    mask_folder,
                                                    actual_fold=fold,
                                                    dataset="validate",
                                                    folds=5)
    testing_set = dataset.ZebrafishDataset_KFold_v2(image_folder,
                                                 mask_folder,
                                                 actual_fold=fold,
                                                 dataset="test",
                                                 folds=5)

    training_loader = torch.utils.data.DataLoader(training_set,
                                                  batch_size=cst.BATCH_SIZE,
                                                  shuffle=True,
                                                  num_workers=cst.WORKERS)

    validation_loader = torch.utils.data.DataLoader(validation_set,
                                                    batch_size=cst.BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=cst.WORKERS)

    testing_loader = torch.utils.data.DataLoader(testing_set,
                                                 batch_size=1,
                                                 shuffle=True,
                                                 num_workers=cst.WORKERS)

    # (Channels x Classes)
    model = UNET(3, 2)
    best_model = UNET(3, 2)
    best_model = model
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    criterion_string = "CE"

    if cst.LOSS == "Dice":
        print("Dice")
        criterion = loss_fn.DiceLoss()
        criterion_string = "DCE"
    if cst.LOSS == "IOU":
        print("IOU")
        criterion = loss_fn.IoULoss()
        criterion_string = "IOU"
    if cst.LOSS == "Tversky":
        print("Twersky")
        criterion = loss_fn.TverskyLoss()
        criterion_string = "Tversky"

    optimiser = torch.optim.Adam(model.parameters(), lr=cst.LEARNING_RATE, weight_decay=cst.WEIGHT_DECAY)
    optimiser_string = "ADAM" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "WD" + str(cst.WEIGHT_DECAY)

    if cst.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=cst.LEARNING_RATE,
                                    momentum=cst.MOMENTUM,
                                    weight_decay=cst.WEIGHT_DECAY)
        optimiser_string = "SGD" + "_" + "LR" + str(cst.LEARNING_RATE) + "_" + "M" + str(cst.MOMENTUM)
        optimiser_string += "_" + "WD" + str(cst.WEIGHT_DECAY)

    model.eval()
    with torch.no_grad():
        val_loss = []
        for images, masks, names in validation_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                vloss = criterion(outputs, masks.to(DEVICE))
            else:
                vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = vloss.detach().item()
            val_loss.append(loss)

        loss = np.mean(val_loss)
        print("Validation loss before training: {}".format(loss))

    best_val = loss
    best_epoch = 0

    params_string = "Params" + "_" + "Epoch" + str(cst.EPOCHS) + "_" + "BS" + str(cst.BATCH_SIZE)
    params_string += "_" + "W" + str(cst.WORKERS)

    epochs_train_losses = []
    epochs_val_losses = []
    for i in range(cst.EPOCHS):
        print("Starting epoch {}".format(i+1), end=". ")

        model.train()
        train_loss = []
        for images, masks, names in training_loader:
            images = transform(images)
            outputs = model(images.to(DEVICE))
            outputs = untransform(outputs)

            masks = masks.type(torch.LongTensor)
            masks = torch.squeeze(masks, 1)

            if cst.LOSS == "CE":
                tloss = criterion(outputs, masks.to(DEVICE))
            else:
                tloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

            loss = tloss.detach().item()
            train_loss.append(loss)

            optimiser.zero_grad()
            tloss.backward()
            optimiser.step()

        loss = np.mean(train_loss)
        epochs_train_losses.append(loss)
        #print("Trained: {}".format(loss), end=". ")

        model.eval()
        with torch.no_grad():
            val_loss = []
            for images, masks, names in validation_loader:
                images = transform(images)
                outputs = model(images.to(DEVICE))
                outputs = untransform(outputs)

                masks = masks.type(torch.LongTensor)
                masks = torch.squeeze(masks, 1)

                if cst.LOSS == "CE":
                    vloss = criterion(outputs, masks.to(DEVICE))
                else:
                    vloss = criterion(outputs, F.one_hot(masks, 2).permute(0, 3, 1, 2).float())

                loss = vloss.detach().item()
                val_loss.append(loss)

            loss = np.mean(val_loss)
            epochs_val_losses.append(loss)
            #print("Validation: {}.".format(loss))

            if loss < best_val:
                best_val = loss
                best_model = model
                best_epoch = i+1

    #print("Training: {}".format(epochs_train_losses))
    #print("Validating: {}".format(epochs_val_losses))
    print("Best score: {}".format(best_val))

    #model_filepath = os.path.join(cst.MODEL, model_name)
    #best_filepath = os.path.join(cst.MODEL, best_name)
    #torch.save(model.state_dict(), model_filepath)
    #torch.save(best_model.state_dict(), best_filepath)

    fold_best_vals.append(best_val)
    
    if best_val < overall_best:
        overall_best = best_val
        overall_best_model = best_model
        best_fold_epoch = best_epoch
        best_fold = fold

    print("--------------------")
    print("Term: " + TERM)
    print("Fold: {}".format(fold))
    print("Last val: {}".format(loss))
    print("Best val: {}".format(best_val))
    print("--------------------")


    
    """index = [i+1 for i in range(cst.EPOCHS)]
    plt.plot(index[1:], epochs_train_losses[1:], label="Training")
    plt.plot(index, epochs_val_losses, label="Validation")
    plt.title(str(cst.LOSS) + "_Fold_" + str(fold)) 
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()"""
    #plot_name = name + ".png"
    #plt.savefig(plot_name)
    

name = TERM + "_" + EXTRA + "_Fold" + str(best_fold) +  "_EPOCH" + str(best_fold_epoch) + "_Val" + str(best_val)
name = name + criterion_string + "_" + optimiser_string + "_" + params_string
model_name = name + ".pth"
best_filepath = os.path.join(cst.MODEL, model_name)
#torch.save(overall_best_model.state_dict(), best_filepath)

print("-------------------------")
print("-----------END-----------")
print("-------------------------")
print("Best fold: {}".format(best_fold+1))
print("Best validation loss: {}".format(overall_best))
print("Best epoch: {}".format(best_fold_epoch))
print("Mean val of folds: {}".format(np.mean(fold_best_vals)))
print("Best vals for each fold:")
a = 0
for item in fold_best_vals:
    print("Fold :{}".format(a+1), end=" - ")
    print("Val: {}".format(item))
    a=a+1
    
    
tps = 0
precisions = []
recalls = []
F1s = []
IOUs = []

eval_model = UNET(3, 2)
eval_model = overall_best_model

eval_model.eval()
for image, mask, name in testing_loader:
    image_name = name[0]
    prediction = predict_img(model, transform(image), DEVICE, untransform, out_threshold=cst.THRESHOLD)
    pred = torch.from_numpy(prediction)
    
    precisions.append(metrics.precision(pred, mask))
    recalls.append(metrics.recall(pred, mask))
    F1s.append(metrics.F1Score(pred, mask))
    IOUs.append(metrics.IOUScore(pred, mask))
    
confidence = 0.9
print("Name of the run: {}".format(TERM + "_" + "Fold_" + str(best_fold)))
print("Precision: {}".format(np.mean(precisions)))
print("90% CI: {}".format(np.percentile(precisions, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(precisions), np.max(precisions))
print()

print("Recall: {}".format(np.mean(recalls)))
print("90% CI: {}".format(np.percentile(recalls, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(recalls), np.max(recalls))
print()
      
print("F1/Dice score: {}".format(np.mean(F1s)))
print("90% CI: {}".format(np.percentile(F1s, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(F1s), np.max(F1s))
print()

print("IoU: {}".format(np.mean(IOUs)))
print("90% CI: {}".format(np.percentile(IOUs, [100*(1-confidence)/2,100*(1-(1-confidence)/2)])))
print("Min, max:", np.min(IOUs), np.max(IOUs))